# Uniprot data scrapping using Gene name 
### Used gene name as a input and stored a GPCR related gene entry in other file 


### Load necessary packages 

In [7]:
from bs4 import BeautifulSoup  # For get data from web
import requests 
import re  # regular expression used for extracting information from txt file 
f = open('input_gene_name_file.txt','r')# get gene list file 

### Function to get Uniprot id using gene name

In [8]:
def get_id_from_gene_file(f):
    """This function gives the Uniprot ID of given gene list in file """
    list1 = []  # empty list we can append the uniprot id of the given gene name that associated with the homo sapiens and that should be a reviewed entry 
    for i in f:
        try: # used for if there is some error accure while running program but that error should not stop the code 
            i.rstrip()  # for removing '\n' from the gene file
            id2 = i.rstrip() 
            # url for fetching information from uniprot with conditional formating 
            # change just url for your interest of data to get the uniprot id of your interest term
            url = ('https://www.uniprot.org/uniprot/?query=reviewed:yes+gene%3A{}+organism%3A%22Homo+sapiens+%28Human%29+%5B9606%5D%22&format=tab&columns=id,reviewed,protein%20names'.format(id2))
            req = requests.get(url)
            soup = req.text
            id1 = re.search(r'Protein names\n(\w.*)\treviewed',soup)  # regular expression for extract only uniprotID 
            uniid = id1.group(1)
            list1.append(uniid) # append ID to list 
    #         print('uniid ='+id1.group(1))
        except Exception as e:  #  if id not match our url this eception handle the error and programe run properly
            pass  # if error occure just go forward and search for another gene name 
    return list1 # return list containg uniprot id 

### Call ID function

In [16]:
gene_id = get_id_from_gene_file(f) # call id function to get the ID list  

### Function for get only frunction of individual  gene from list of gene 

In [17]:
def get_function(gene_id):
    functions = []   # empty list for the store function of that gene 
    for i in gene_id:
        url = ('https://www.uniprot.org/uniprot/{}').format(i)  # get html page of indivisual entry
        req = requests.get(url).text    # req object for extract informaton in text format
        soup = BeautifulSoup(req,'lxml')  # soup objetc content html data
        id1 = soup.find('span',{'property':'text'}).text  # extract only span tag which contains the function of the given gene 
        fun = re.search('(.+)',id1) # regular expresion for matching text of function  
        fun = fun.group(0) # it contains the match string 
        functions.append(fun)  # append in list 
    return functions  # return only list of function 

### Call  get_function for creating list of function  

In [18]:
functions = get_function(gene_id)  # list of functions of given uniprot id 

### Dictionary that stored the function and associated uniprot id 

In [22]:
# using zip() 
# to convert lists to dictionary
# assign each function with there uniprot id 
dict1 = dict(zip(functions,gene_id)) 

### function for check the indivisul id which contains the GPCR related activity or not

In [19]:
def validated_gene(dict1): # this function for vaidate the gene that 
    validated_id = []
    for key, value in dict1.items():
        # input your interst of keyword to validate the gene that are related to this function 
        g_protein = ['G-protein coupled receptor','G-proteins','GPCR','G proteins','G protein','G protein coupled receptor signaling pathway','G protein coupled receptor pathway','G protein coupled receptor activity']
        for j in g_protein:
            if j in key :
                validated_id.append(value) # append validated id which contains the gpcr activity 
    return validated_id

## Validated Uniprot Id 

In [21]:
validated_id = validated_gene(dict1)
uniquie_id = set(validated_id) # append in set for get the unique list of entry 
uniquie_id = list(uniquie_id) # list containg gpcr activity 